In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import sys
import librosa
import numpy as np
import os
import glob
import json

import torch
from text.multi_apha import text_to_sequence
from text.multi_apha import letter_
import commons
import attentions
import modules
import models
import utils



In [2]:
def plot_waveform(waveform, sr, title="Waveform"):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sr

    figure, axes = plt.subplots(num_channels, 1)
    axes.plot(time_axis, waveform[0], linewidth=1)
    axes.grid(True)
    figure.suptitle(title)
    plt.show(block=False)


def plot_spectrogram(specgram, title=None, ylabel="freq_bin"):
    fig, axs = plt.subplots(1, 1)
    axs.set_title(title or "Spectrogram (db)")
    axs.set_ylabel(ylabel)
    axs.set_xlabel("frame")
    im = axs.imshow(librosa.power_to_db(specgram), origin="lower", aspect="auto")
    fig.colorbar(im, ax=axs)
    plt.show(block=False)

In [3]:
# If you are using your own trained model
model_dir = "/media/caijb/data_drive/glowtts_KR/logs/cross"
hps = utils.get_hparams_from_dir(model_dir)
checkpoint_path = utils.latest_checkpoint_path(model_dir)

# If you are using a provided pretrained model
# hps = utils.get_hparams_from_file("./configs/any_config_file.json")
# checkpoint_path = "/path/to/pretrained_model"

model = models.FlowGenerator(
    len(letter_) + getattr(hps.data, "add_blank", False),
    out_channels=hps.data.n_mel_channels,n_speakers=hps.model.n_speaker
    ,gin_channels=512,
    **hps.model).to("cuda")

utils.load_checkpoint(checkpoint_path, model)
model.decoder.store_inverse() # do not calcuate jacobians for fast decoding
_ = model.eval()


# normalizing & type casting
def normalize_audio(x, max_wav_value=hps.data.max_wav_value):
    return np.clip((x / np.abs(x).max()) * max_wav_value, -32768, 32767).astype("int32")

/media/caijb/data_drive/glowtts_KR/logs/cross/G_150.pth


/home/caijb/Desktop/GlowTTS_KR/modules.py:205: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2491.)
  w_init = torch.qr(torch.FloatTensor(self.n_split, self.n_split).normal_())[0]


INFO:root:Loaded checkpoint '/media/caijb/data_drive/glowtts_KR/logs/cross/G_150.pth' (iteration 150)


In [26]:
tst_stn = "Dieser Satz wird nicht zum Lernen verwendet."
if getattr(hps.data, "add_blank", True):
    text_norm = text_to_sequence(tst_stn.strip(), 'english')
    text_norm = commons.intersperse(text_norm, len(letter_))
else: # If not using "add_blank" option during training, adding spaces at the beginning and the end of utterance improves quality
    tst_stn = " " + tst_stn.strip() + " "
    text_norm = text_to_sequence(tst_stn.strip(), 'english')
print(text_norm)
sequence = np.array(text_norm)[None, :]
print("".join([letter_[c] if c < len(letter_) else "<BNK>" for c in sequence[0]]))
x_tst = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
x_tst_lengths = torch.tensor([x_tst.shape[1]]).cuda()

[67, 44, 67, 49, 67, 45, 67, 59, 67, 45, 67, 58, 67, 14, 67, 59, 67, 41, 67, 60, 67, 66, 67, 14, 67, 63, 67, 49, 67, 58, 67, 44, 67, 14, 67, 54, 67, 49, 67, 43, 67, 48, 67, 60, 67, 14, 67, 66, 67, 61, 67, 53, 67, 14, 67, 52, 67, 45, 67, 58, 67, 54, 67, 45, 67, 54, 67, 14, 67, 62, 67, 45, 67, 58, 67, 63, 67, 45, 67, 54, 67, 44, 67, 45, 67, 60, 67, 8, 67]
<BNK>d<BNK>i<BNK>e<BNK>s<BNK>e<BNK>r<BNK> <BNK>s<BNK>a<BNK>t<BNK>z<BNK> <BNK>w<BNK>i<BNK>r<BNK>d<BNK> <BNK>n<BNK>i<BNK>c<BNK>h<BNK>t<BNK> <BNK>z<BNK>u<BNK>m<BNK> <BNK>l<BNK>e<BNK>r<BNK>n<BNK>e<BNK>n<BNK> <BNK>v<BNK>e<BNK>r<BNK>w<BNK>e<BNK>n<BNK>d<BNK>e<BNK>t<BNK>.<BNK>


In [28]:
with torch.no_grad():
    noise_scale = .667
    length_scale = 1.0
    sid=[8]
    #dutch 0, english 1, franch 2, spanish 3
    lang=[0]
    sid=torch.tensor(sid).cuda()
    lang=torch.tensor(lang).cuda()
    (y_gen, *_), *_, (attn_gen, *_) = model(x_tst, x_tst_lengths,g=sid,l=lang, gen=True)
    if not os.path.exists('./hifi-gan/test_mel_files'):
        os.makedirs('./hifi-gan/test_mel_files')
    np.save("./hifi-gan/test_mel_files/EN2German.npy", y_gen.cpu().detach().numpy())
    import audio_processing as ap
    y_gen=ap.dynamic_range_decompression(y_gen)
    mel=y_gen.detach().cpu()
    mel=mel.numpy()
    mel_basis=librosa.filters.mel(sr=hps.data.sampling_rate, n_fft=hps.data.filter_length, n_mels=hps.data.n_mel_channels)
    covered_mel=librosa.util.nnls(mel_basis, mel)
    audio=librosa.griffinlim(covered_mel,n_iter=60)
    #stft_fn=STFT(hps.data.filter_length, hps.data.hop_length, hps.data.win_length)
    #audio=ap.griffin_lim(torch.tensor(covered_mel),stft_fn,n_iters=60)

#fig, ax = plt.subplots(nrows=2, sharex=True, sharey=True)
#import librosa.display as dp
#dp.waveplot(audio, sr=hps.data.sampling_rate, color='b', ax=ax[0])
#ax[0].set(title='Original', xlabel=None)
#ax[0].label_outer()
ipd.Audio(audio, rate=hps.data.sampling_rate)